In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from numpy import concatenate
dataset = pd.read_csv('gridWeather_201804.csv', header=0, index_col=0)
dataset.head(5)

Using TensorFlow backend.


,station_id,time,weather,temperature,pressure,humidity,wind_direction,wind_speed
id,,,,,,,,
2027019,beijing_grid_000,2018-04-01 00:00:00,CLEAR_DAY,15.0,974.1536,21.0,346.93,4.18
2027020,beijing_grid_001,2018-04-01 00:00:00,CLEAR_DAY,15.0,960.6752,20.0,329.47,4.50
2027021,beijing_grid_002,2018-04-01 00:00:00,CLEAR_DAY,9.0,947.1968,19.0,315.38,5.16
2027022,beijing_grid_003,2018-04-01 00:00:00,CLEAR_DAY,9.0,931.8720,19.0,307.81,5.74
2027023,beijing_grid_004,2018-04-01 00:00:00,CLEAR_DAY,9.0,914.7008,20.0,304.23,6.08


In [2]:
dataset.isnull().any()

station_id        False
time              False
weather           False
temperature       False
pressure          False
humidity          False
wind_direction    False
wind_speed        False
dtype: bool

In [3]:
near_stations = {'aotizhongxin_aq': 'beijing_grid_304',
 'badaling_aq': 'beijing_grid_224',
 'beibuxinqu_aq': 'beijing_grid_263',
 'daxing_aq': 'beijing_grid_301',
 'dingling_aq': 'beijing_grid_265',
 'donggaocun_aq': 'beijing_grid_452',
 'dongsi_aq': 'beijing_grid_303',
 'dongsihuan_aq': 'beijing_grid_324',
 'fangshan_aq': 'beijing_grid_238',
 'fengtaihuayuan_aq': 'beijing_grid_282',
 'guanyuan_aq': 'beijing_grid_282',
 'gucheng_aq': 'beijing_grid_261',
 'huairou_aq': 'beijing_grid_349',
 'liulihe_aq': 'beijing_grid_216',
 'mentougou_aq': 'beijing_grid_240',
 'miyun_aq': 'beijing_grid_392',
 'miyunshuiku_aq': 'beijing_grid_414',
 'nansanhuan_aq': 'beijing_grid_303',
 'nongzhanguan_aq': 'beijing_grid_324',
 'pingchang_aq': 'beijing_grid_264',
 'pinggu_aq': 'beijing_grid_452',
 'qianmen_aq': 'beijing_grid_303',
 'shunyi_aq': 'beijing_grid_368',
 'tiantan_aq': 'beijing_grid_303',
 'tongzhou_aq': 'beijing_grid_366',
 'wanliu_aq': 'beijing_grid_283',
 'wanshouxigong_aq': 'beijing_grid_303',
 'xizhimenbei_aq': 'beijing_grid_283',
 'yanqin_aq': 'beijing_grid_225',
 'yizhuang_aq': 'beijing_grid_323',
 'yongdingmennei_aq': 'beijing_grid_303',
 'yongledian_aq': 'beijing_grid_385',
 'yufa_aq': 'beijing_grid_278',
 'yungang_aq': 'beijing_grid_239',
 'zhiwuyuan_aq': 'beijing_grid_262'}

In [4]:
#LSTM数据准备
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [5]:
dataset.drop('weather', axis=1, inplace=True)
dataset.head(5)

,station_id,time,temperature,pressure,humidity,wind_direction,wind_speed
id,,,,,,,
2027019,beijing_grid_000,2018-04-01 00:00:00,15.0,974.1536,21.0,346.93,4.18
2027020,beijing_grid_001,2018-04-01 00:00:00,15.0,960.6752,20.0,329.47,4.50
2027021,beijing_grid_002,2018-04-01 00:00:00,9.0,947.1968,19.0,315.38,5.16
2027022,beijing_grid_003,2018-04-01 00:00:00,9.0,931.8720,19.0,307.81,5.74
2027023,beijing_grid_004,2018-04-01 00:00:00,9.0,914.7008,20.0,304.23,6.08


In [6]:
labels=pd.read_csv('aiqQuality_201804.csv', header=0, index_col=0)
labels.head(5)

,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
id,,,,,,,,
2942426,dongsi_aq,2018-04-01 02:00:00,259.0,NaN,98.0,3.4,56.0,57.0
2942427,tiantan_aq,2018-04-01 02:00:00,250.0,NaN,95.0,3.1,64.0,28.0
2942428,guanyuan_aq,2018-04-01 02:00:00,240.0,246.0,107.0,2.2,49.0,30.0
2942429,wanshouxigong_aq,2018-04-01 02:00:00,255.0,260.0,87.0,2.5,65.0,35.0
2942430,aotizhongxin_aq,2018-04-01 02:00:00,266.0,NaN,111.0,3.5,45.0,57.0


In [7]:
pred =  pd.read_csv('gridWeather_20180501-20180502.csv', header=0)
pred.head(5)

,id,station_id,time,weather,temperature,pressure,humidity,wind_direction,wind_speed
0,3116510,beijing_grid_000,2018-05-01 00:00:00,CLOUDY,20.0,975.6105,52.0,76.64,4.38
1,3116511,beijing_grid_001,2018-05-01 00:00:00,CLOUDY,20.0,962.0873,49.0,58.35,3.82
2,3116512,beijing_grid_002,2018-05-01 00:00:00,CLOUDY,13.0,948.5641,46.0,36.82,3.75
3,3116513,beijing_grid_003,2018-05-01 00:00:00,CLOUDY,13.0,933.2105,43.0,9.52,4.65
4,3116514,beijing_grid_004,2018-05-01 00:00:00,CLOUDY,13.0,916.0265,42.0,350.17,6.88


In [18]:
near_stations1 = {'aotizhongxin_aq': 'beijing_grid_304',
 'badaling_aq': 'beijing_grid_224',
 'beibuxinqu_aq': 'beijing_grid_263',
 'daxing_aq': 'beijing_grid_301',
 'dingling_aq': 'beijing_grid_265',
 'donggaocun_aq': 'beijing_grid_452',
 'dongsi_aq': 'beijing_grid_303',
 'dongsihuan_aq': 'beijing_grid_324',
 'fangshan_aq': 'beijing_grid_238',
 'fengtaihuayuan_aq': 'beijing_grid_282',
 'guanyuan_aq': 'beijing_grid_282',
 'gucheng_aq': 'beijing_grid_261',
 'huairou_aq': 'beijing_grid_349',
 'liulihe_aq': 'beijing_grid_216',
 'mentougou_aq': 'beijing_grid_240',
 'miyun_aq': 'beijing_grid_392',
 'miyunshuiku_aq': 'beijing_grid_414',
 'nansanhuan_aq': 'beijing_grid_303',
 'nongzhanguan_aq': 'beijing_grid_324',
 'pingchang_aq': 'beijing_grid_264',
 'pinggu_aq': 'beijing_grid_452',
 'qianmen_aq': 'beijing_grid_303',
 'shunyi_aq': 'beijing_grid_368',
 'tiantan_aq': 'beijing_grid_303',
 'tongzhou_aq': 'beijing_grid_366',
 'wanliu_aq': 'beijing_grid_283',
 'wanshouxigong_aq': 'beijing_grid_303',
 'xizhimenbei_aq': 'beijing_grid_283',
 'yanqin_aq': 'beijing_grid_225',
 'yizhuang_aq': 'beijing_grid_323',
 'yongdingmennei_aq': 'beijing_grid_303',
 'yongledian_aq': 'beijing_grid_385',
 'yufa_aq': 'beijing_grid_278',
 'yungang_aq': 'beijing_grid_239'}

In [20]:
table_pm25=[]
for i in near_stations1.keys():
    #if near_stations[i]=='beijing_grid_304':
    print(i)
    #beijing_grid_304(aotizhongxin_aq)
    dataset1=dataset.loc[dataset.station_id==near_stations[i]]
    print('dataset:',len(dataset1))
    labels1=labels.loc[labels.station_id==i]
    print('labels:',len(labels1))
    fullData=pd.merge(dataset1, labels1, on = 'time')
    dataset1=fullData[['PM25_Concentration','temperature','pressure','humidity','wind_direction','wind_speed']]
    values = dataset1.values
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scalerfit = scaler.fit(values)
    scaled = scalerfit.transform(values)
    # frame as supervised learning
    #把下一个label提上来做label用
    reframed = series_to_supervised(scaled, 1, 1)
    # drop columns we don't want to predict
    reframed.drop(reframed.columns[[7,8,9,10,11]], axis=1, inplace=True)
    values1 = reframed.values
    n_train_hours = 20 * 24
    train = values1[:n_train_hours, :]
    test = values1[n_train_hours:, :]
    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    # design network
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)


    #读入预测数据
    pred1=pred.loc[pred.station_id==near_stations[i]]
    pred1.drop(['id','station_id','time','weather'], axis=1, inplace=True)
    #加一个变量以匹配数据归一化格式，在后面预测时会去掉
    pred1['PM25_Concentration']=10
    pred2=pred1.reindex(columns=list(['PM25_Concentration','temperature','pressure','humidity','wind_direction','wind_speed']))
    pred2 =scalerfit.transform(pred2)
    a=np.append(test_y[-1],pred2[0:1,1:])
    a=a.reshape(1,1,6)
    b = model.predict(a)
    prediction=[]
    k=0
    for j in range(len(pred2)):            
        a=np.append(b,pred2[j:j+1,1:])
        a=a.reshape(1,1,6)
        b = model.predict(a)    
        prediction.append(b)
    print(len(prediction))
    test3=pred2[:,1:]
    df2 = np.resize(prediction, len(prediction)).reshape(len(prediction),1)
    # invert scaling for forecast
    inv_yhat = concatenate((df2, test3), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    df = pd.DataFrame(inv_yhat,columns=['pm2.5'])    
    df['new']=df.apply(lambda x: x.index)
    df['test_id']=i+'#'
    df['test_id'] = df['test_id']+ df['new'].astype(str) 
    df.drop('new',axis=1,inplace=True)
    df1=df.reindex(columns=list(['test_id','pm2.5']))
    df1.to_csv('d:/data/'+i+'.csv')
    table_pm25.append(df1)
    #print(df)
    if near_stations[i]=='beijing_grid_304':
        final_labels=pd.read_csv('d:/data/final_labels.csv',header=0)
        def smape2(actual, predicted):
            dividend= np.abs(np.array(actual) - np.array(predicted))
            c = np.array(actual) + np.array(predicted)
            denominator= np.array(actual) + np.array(predicted)
            return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))
        smape2=smape2(final_labels, inv_yhat)                        
        print(i+' smape:',smape2)

        

aotizhongxin_aq
dataset: 711
labels: 666
Train on 480 samples, validate on 171 samples
Epoch 1/50
 - 10s - loss: 0.2051 - val_loss: 0.1791
Epoch 2/50
 - 0s - loss: 0.1827 - val_loss: 0.1604
Epoch 3/50
 - 0s - loss: 0.1738 - val_loss: 0.1506
Epoch 4/50
 - 0s - loss: 0.1685 - val_loss: 0.1451
Epoch 5/50
 - 0s - loss: 0.1640 - val_loss: 0.1412
Epoch 6/50
 - 0s - loss: 0.1592 - val_loss: 0.1376
Epoch 7/50
 - 0s - loss: 0.1541 - val_loss: 0.1339
Epoch 8/50
 - 0s - loss: 0.1492 - val_loss: 0.1301
Epoch 9/50
 - 0s - loss: 0.1443 - val_loss: 0.1260
Epoch 10/50
 - 0s - loss: 0.1395 - val_loss: 0.1219
Epoch 11/50
 - 0s - loss: 0.1345 - val_loss: 0.1181
Epoch 12/50
 - 0s - loss: 0.1296 - val_loss: 0.1144
Epoch 13/50
 - 0s - loss: 0.1246 - val_loss: 0.1107
Epoch 14/50
 - 0s - loss: 0.1196 - val_loss: 0.1071
Epoch 15/50
 - 0s - loss: 0.1145 - val_loss: 0.1039
Epoch 16/50
 - 0s - loss: 0.1092 - val_loss: 0.1012
Epoch 17/50
 - 0s - loss: 0.1041 - val_loss: 0.0983
Epoch 18/50
 - 0s - loss: 0.0994 - va

C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


48
aotizhongxin_aq smape: 0.6941308550864226
badaling_aq
dataset: 712
labels: 666
Train on 480 samples, validate on 137 samples
Epoch 1/50
 - 9s - loss: 0.1936 - val_loss: 0.1238
Epoch 2/50
 - 0s - loss: 0.1681 - val_loss: 0.0944
Epoch 3/50
 - 0s - loss: 0.1636 - val_loss: 0.0875
Epoch 4/50
 - 0s - loss: 0.1610 - val_loss: 0.0852
Epoch 5/50
 - 0s - loss: 0.1566 - val_loss: 0.0842
Epoch 6/50
 - 0s - loss: 0.1520 - val_loss: 0.0832
Epoch 7/50
 - 0s - loss: 0.1478 - val_loss: 0.0816
Epoch 8/50
 - 0s - loss: 0.1435 - val_loss: 0.0787
Epoch 9/50
 - 0s - loss: 0.1390 - val_loss: 0.0755
Epoch 10/50
 - 0s - loss: 0.1345 - val_loss: 0.0728
Epoch 11/50
 - 0s - loss: 0.1299 - val_loss: 0.0703
Epoch 12/50
 - 0s - loss: 0.1252 - val_loss: 0.0680
Epoch 13/50
 - 0s - loss: 0.1202 - val_loss: 0.0658
Epoch 14/50
 - 0s - loss: 0.1152 - val_loss: 0.0639
Epoch 15/50
 - 0s - loss: 0.1101 - val_loss: 0.0623
Epoch 16/50
 - 0s - loss: 0.1050 - val_loss: 0.0611
Epoch 17/50
 - 0s - loss: 0.1000 - val_loss: 0.05

C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


48
beibuxinqu_aq
dataset: 712
labels: 666
Train on 480 samples, validate on 165 samples
Epoch 1/50
 - 10s - loss: 0.1551 - val_loss: 0.0897
Epoch 2/50
 - 0s - loss: 0.1359 - val_loss: 0.0824
Epoch 3/50
 - 0s - loss: 0.1323 - val_loss: 0.0802
Epoch 4/50
 - 0s - loss: 0.1285 - val_loss: 0.0770
Epoch 5/50
 - 0s - loss: 0.1240 - val_loss: 0.0737
Epoch 6/50
 - 0s - loss: 0.1199 - val_loss: 0.0707
Epoch 7/50
 - 0s - loss: 0.1158 - val_loss: 0.0682
Epoch 8/50
 - 0s - loss: 0.1115 - val_loss: 0.0661
Epoch 9/50
 - 0s - loss: 0.1072 - val_loss: 0.0643
Epoch 10/50
 - 0s - loss: 0.1030 - val_loss: 0.0626
Epoch 11/50
 - 0s - loss: 0.0991 - val_loss: 0.0612
Epoch 12/50
 - 0s - loss: 0.0955 - val_loss: 0.0597
Epoch 13/50
 - 0s - loss: 0.0920 - val_loss: 0.0585
Epoch 14/50
 - 0s - loss: 0.0887 - val_loss: 0.0576
Epoch 15/50
 - 0s - loss: 0.0854 - val_loss: 0.0565
Epoch 16/50
 - 0s - loss: 0.0822 - val_loss: 0.0551
Epoch 17/50
 - 0s - loss: 0.0791 - val_loss: 0.0540
Epoch 18/50
 - 0s - loss: 0.0760 - v

...

48
yufa_aq
dataset: 712
labels: 666
Train on 480 samples, validate on 154 samples
Epoch 1/50
 - 17s - loss: 0.1803 - val_loss: 0.1766
Epoch 2/50
 - 0s - loss: 0.1507 - val_loss: 0.1390
Epoch 3/50
 - 0s - loss: 0.1424 - val_loss: 0.1247
Epoch 4/50
 - 0s - loss: 0.1395 - val_loss: 0.1196
Epoch 5/50
 - 0s - loss: 0.1367 - val_loss: 0.1172
Epoch 6/50
 - 0s - loss: 0.1335 - val_loss: 0.1158
Epoch 7/50
 - 0s - loss: 0.1301 - val_loss: 0.1144
Epoch 8/50
 - 0s - loss: 0.1268 - val_loss: 0.1126
Epoch 9/50
 - 0s - loss: 0.1235 - val_loss: 0.1100
Epoch 10/50
 - 0s - loss: 0.1201 - val_loss: 0.1067
Epoch 11/50
 - 0s - loss: 0.1164 - val_loss: 0.1030
Epoch 12/50
 - 0s - loss: 0.1126 - val_loss: 0.0990
Epoch 13/50
 - 0s - loss: 0.1086 - val_loss: 0.0951
Epoch 14/50
 - 0s - loss: 0.1044 - val_loss: 0.0910
Epoch 15/50
 - 0s - loss: 0.1000 - val_loss: 0.0871
Epoch 16/50
 - 0s - loss: 0.0955 - val_loss: 0.0833
Epoch 17/50
 - 0s - loss: 0.0911 - val_loss: 0.0797
Epoch 18/50
 - 0s - loss: 0.0867 - val_los

C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


48
yungang_aq
dataset: 712
labels: 666
Train on 480 samples, validate on 122 samples
Epoch 1/50
 - 17s - loss: 0.1923 - val_loss: 0.2008
Epoch 2/50
 - 0s - loss: 0.1607 - val_loss: 0.1648
Epoch 3/50
 - 0s - loss: 0.1515 - val_loss: 0.1512
Epoch 4/50
 - 0s - loss: 0.1474 - val_loss: 0.1454
Epoch 5/50
 - 0s - loss: 0.1441 - val_loss: 0.1422
Epoch 6/50
 - 0s - loss: 0.1405 - val_loss: 0.1398
Epoch 7/50
 - 0s - loss: 0.1368 - val_loss: 0.1377
Epoch 8/50
 - 0s - loss: 0.1331 - val_loss: 0.1351
Epoch 9/50
 - 0s - loss: 0.1294 - val_loss: 0.1322
Epoch 10/50
 - 0s - loss: 0.1256 - val_loss: 0.1285
Epoch 11/50
 - 0s - loss: 0.1215 - val_loss: 0.1245
Epoch 12/50
 - 0s - loss: 0.1173 - val_loss: 0.1201
Epoch 13/50
 - 0s - loss: 0.1130 - val_loss: 0.1154
Epoch 14/50
 - 0s - loss: 0.1085 - val_loss: 0.1109
Epoch 15/50
 - 0s - loss: 0.1040 - val_loss: 0.1064
Epoch 16/50
 - 0s - loss: 0.0995 - val_loss: 0.1019
Epoch 17/50
 - 0s - loss: 0.0952 - val_loss: 0.0976
Epoch 18/50
 - 0s - loss: 0.0910 - val_

C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


48


In [165]:
DataFrame(table_pm25).to_csv('table_pm25.csv')

In [15]:
table_pm25

[               test_id      pm2.5
 0    aotizhongxin_aq#0  66.546390
 1    aotizhongxin_aq#1  61.264441
 2    aotizhongxin_aq#2  55.224316
 3    aotizhongxin_aq#3  49.784989
 4    aotizhongxin_aq#4  44.884459
 5    aotizhongxin_aq#5  41.608490
 6    aotizhongxin_aq#6  40.600734
 7    aotizhongxin_aq#7  42.266281
 8    aotizhongxin_aq#8  45.633423
 9    aotizhongxin_aq#9  50.002015
 10  aotizhongxin_aq#10  55.950968
 11  aotizhongxin_aq#11  57.112316
 12  aotizhongxin_aq#12  58.150014
 13  aotizhongxin_aq#13  59.559590
 14  aotizhongxin_aq#14  61.731374
 15  aotizhongxin_aq#15  62.339803
 16  aotizhongxin_aq#16  64.665461
 17  aotizhongxin_aq#17  65.089402
 18  aotizhongxin_aq#18  64.897740
 19  aotizhongxin_aq#19  63.119053
 20  aotizhongxin_aq#20  60.255310
 21  aotizhongxin_aq#21  56.853210
 22  aotizhongxin_aq#22  51.302855
 23  aotizhongxin_aq#23  44.476449
 24  aotizhongxin_aq#24  36.798942
 25  aotizhongxin_aq#25  30.289477
 26  aotizhongxin_aq#26  24.398334
 27  aotizhongxin_aq

In [21]:
fullData=pd.DataFrame(columns=['test_id','pm2.5'])
for i in near_stations1.keys():
    train=pd.read_csv('data/'+i+'.csv',header=0,index_col=0)
    fullData =pd.concat([fullData,train],axis=0)
fullData

,test_id,pm2.5
0,aotizhongxin_aq#0,68.522365
1,aotizhongxin_aq#1,64.416760
2,aotizhongxin_aq#2,59.264796
3,aotizhongxin_aq#3,54.480861
4,aotizhongxin_aq#4,50.522772
5,aotizhongxin_aq#5,48.395860
6,aotizhongxin_aq#6,48.094276
7,aotizhongxin_aq#7,50.217162
8,aotizhongxin_aq#8,54.407348
9,aotizhongxin_aq#9,59.147836


In [22]:
fullData.to_csv('reult.csv')